In [54]:
import pandas as pd

In [6]:
pd_test = pd.read_csv("test_mul.csv",dtype="str",names=["a","arxiv_id","create_date","update_date","cats","subcats","title","abstract"]).iloc[1:,1:]

In [52]:
pd_train = pd.read_csv("train_mul.csv",dtype="str",names=["arxiv_id","create_date","update_date","cats","subcats","title","abstract"])

In [53]:
pd_train.shape

(91760, 7)

Clean up abstract using noun only phrase

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Volumes/HD-500GB/Users/nikolausn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Volumes/HD-500GB/Users/nikolausn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Volumes/HD-500GB/Users/nikolausn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
def noun_only_sentence(x_string):
    string_list = x_string.lower().split(".")
    noun_sentence = []
    for my_str in string_list:
        #print(my_str)
        tokens = word_tokenize(my_str)
        #bigrams = ngrams(token,2)
        # remove all tokens that are not alphabetic
        is_noun = lambda pos: pos[:2] == 'NN'
        # do the nlp stuff
        nouns = [word for (word, pos) in nltk.pos_tag(tokens) if is_noun(pos)] 
        noun_sentence.append(nouns)
    return noun_sentence

In [12]:
#abstract_noun = pd_train.abstract.apply(lambda x: noun_only_sentence(x))

In [13]:
#abstract_noun_test = pd_test.abstract.apply(lambda x: noun_only_sentence(x))

In [15]:
import pickle
#with open("/gdrive/My Drive/Dataset/arxiv/abstract_noun.pickle","wb") as file:
#  pickle.dump((abstract_noun,abstract_noun_test),file)
with open("abstract_noun.pickle","rb") as file:
  (abstract_noun,abstract_noun_test) = pickle.load(file)

In [16]:
# clean abstract noun so it will contain only alphabetic char

# use english stop_words
stop_words =  list(stopwords.words('english'))

def alphabetic_only_l(x_string):
  list_sent = []    
  for tokens in x_string:
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha() and word not in stop_words and len(word)>3]
    list_sent.append(words)    
  return list_sent

In [ ]:
#abs_noun_clean = abstract_noun.apply(lambda x:alphabetic_only_l(x))

In [ ]:
#abs_noun_clean_test = abstract_noun_test.apply(lambda x:alphabetic_only_l(x))

In [19]:
#with open("/gdrive/My Drive/Dataset/arxiv/abstract_noun_clean.pickle","wb") as file:
#  pickle.dump((abs_noun_clean,abs_noun_clean_test),file)
with open("abstract_noun_clean.pickle","rb") as file:
  (abs_noun_clean,abs_noun_clean_test) = pickle.load(file)

In [20]:
# features/words chooser using tfidf
def sent_convert(x_string):
  list_sent = []
  for sent in x_string:
    # remove all tokens that are not alphabetic    
    list_sent.append(" ".join(sent))    
  return ". ".join(list_sent)

abs_noun_sent = abs_noun_clean.apply(lambda x:sent_convert(x))

In [21]:
abs_noun_sent_test = abs_noun_clean_test.apply(lambda x:sent_convert(x))

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
tf = tf_vectorizer.fit_transform(abs_noun_sent.values)
tf_feature_names = tf_vectorizer.get_feature_names()

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf = tfidf_vectorizer.fit_transform(abs_noun_sent.values)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

/Volumes/HD-500GB/Users/nikolausn/Applications/anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [23]:
len(tfidf_feature_names)

27752

In [24]:
tfidf_sum = tfidf.sum(axis=0)

In [25]:
tfidf_sum.shape

(1, 27752)

In [26]:
import numpy as np
# sort the tfidf_desc
index_sort = np.argsort(tfidf_sum)[:,::-1]

In [27]:
index_sort

matrix([[15950,  5928, 21405, ...,   629, 13003,  3128]])

In [28]:
pot_feat = [tfidf_feature_names[x] for x in index_sort.tolist()[0][:100]]

In [29]:
pot_feat

['model',
 'data',
 'results',
 'theory',
 'energy',
 'field',
 'models',
 'system',
 'method',
 'quantum',
 'time',
 'states',
 'state',
 'mass',
 'systems',
 'paper',
 'order',
 'function',
 'phase',
 'equation',
 'approach',
 'problem',
 'analysis',
 'number',
 'space',
 'case',
 'structure',
 'properties',
 'equations',
 'matter',
 'solutions',
 'dynamics',
 'functions',
 'distribution',
 'density',
 'parameters',
 'collisions',
 'lattice',
 'symmetry',
 'effects',
 'methods',
 'terms',
 'information',
 'network',
 'limit',
 'production',
 'transition',
 'framework',
 'work',
 'measurements',
 'temperature',
 'process',
 'solution',
 'algorithm',
 'conditions',
 'parameter',
 'fields',
 'interaction',
 'interactions',
 'networks',
 'result',
 'form',
 'study',
 'effect',
 'physics',
 'decay',
 'gauge',
 'range',
 'experiments',
 'simulations',
 'potential',
 'values',
 'particles',
 'particle',
 'gravity',
 'quark',
 'value',
 'momentum',
 'point',
 'rate',
 'theories',
 'spectrum'

In [30]:
pd_train.head()

,arxiv_id,create_date,update_date,cats,subcats,title,abstract
0,1508.05418,2015-08-21,NaN,math,math.AT|math.AG|math.CO|math.RA,The Cohomology of Quaternionic Hyperplane Comp...,"Over the complex numbers, the complement of a ..."
1,1411.6304,2014-11-23,NaN,math-ph|math,math-ph|math.MP,Dephasing of Kuramoto oscillators in kinetic r...,We study the kinetic Kuramoto model for couple...
2,1402.0313,2014-02-03,NaN,cs|math,cs.IT|math.IT,Rate-Distortion Properties of Single-Layer Qua...,The Quantize & Forward (QF) scheme for two-way...
3,1611.07471,2016-11-22,2018-04-04,math,math.AG,Distinguished models of intermediate Jacobians,We show that the image of the Abel-Jacobi map ...
4,1806.00355,2018-05-31,NaN,math,math.HO|math.NT,Mahler's work on Diophantine equations and sub...,We discuss Mahler's work on Diophantine approx...


extracting the class

In [31]:
y_train_label = pd_train.cats.apply(lambda x: x.split("|")[0])

In [32]:
y_train_label.unique()

array(['math', 'math-ph', 'cs', 'quant-ph', 'hep-th', 'stat', 'cond-mat',
       'q-fin', 'nlin', 'physics', 'gr-qc', 'q-bio', 'eess', 'astro-ph',
       'nucl-th', 'econ', 'hep-ph', 'hep-lat', 'nucl-ex', 'hep-ex'],
      dtype=object)

In [33]:
y_train_class = y_train_label.apply(lambda x:list(y_train_label.unique()).index(x))

In [34]:
y_test_class = pd_test.cats.apply(lambda x: x.split("|")[0]).apply(lambda x:list(y_train_label.unique()).index(x))

**Logistic Regression Model over TF-IDF vector**

In [35]:
num_features = 10000

In [36]:
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
X_train = tfidf[:,index_sort.tolist()[0][:num_features]]
y_train = y_train_class.values
lr_cv = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_train, y_train)

In [37]:
X_test = tfidf_vectorizer.transform(abs_noun_sent_test.values)[:,index_sort.tolist()[0][:num_features]]

In [38]:
X_test.shape

(18942, 10000)

In [39]:
y_test_pred = lr_cv.predict(X_test)

Performance Metrics

In [40]:
from sklearn.metrics import classification_report,accuracy_score,cohen_kappa_score
acc = accuracy_score(y_test_class,y_test_pred)
kap = cohen_kappa_score(y_test_class,y_test_pred)
print("accuracy:",acc)
print("kappa:",kap)
print(classification_report(y_test_class,y_test_pred))

accuracy: 0.6944884383908774
kappa: 0.6734105587931851
             precision    recall  f1-score   support

          0       0.62      0.74      0.68      1593
          1       0.48      0.34      0.40       644
          2       0.63      0.75      0.69      1547
          3       0.75      0.69      0.72      1009
          4       0.67      0.71      0.69      1118
          5       0.59      0.58      0.59       667
          6       0.71      0.75      0.73      1389
          7       0.85      0.86      0.86       998
          8       0.62      0.56      0.59       550
          9       0.59      0.61      0.60      1539
         10       0.71      0.67      0.69       881
         11       0.68      0.69      0.68       778
         12       0.55      0.24      0.33       363
         13       0.85      0.81      0.83      1321
         14       0.72      0.68      0.70       926
         15       0.75      0.05      0.09        66
         16       0.74      0.80      0.77 

**Random Forest**

In [41]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=500,random_state=0).fit(X_train, y_train)

/Volumes/HD-500GB/Users/nikolausn/Applications/anaconda/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [42]:
from sklearn.metrics import classification_report,accuracy_score,cohen_kappa_score
y_test_pred = rf_model.predict(X_test)
acc = accuracy_score(y_test_class,y_test_pred)
kap = cohen_kappa_score(y_test_class,y_test_pred)
print("accuracy:",acc)
print("kappa:",kap)
print(classification_report(y_test_class,y_test_pred))

accuracy: 0.6898426776475557
kappa: 0.6676397886425085
             precision    recall  f1-score   support

          0       0.54      0.77      0.63      1593
          1       0.75      0.25      0.38       644
          2       0.55      0.82      0.66      1547
          3       0.73      0.73      0.73      1009
          4       0.66      0.68      0.67      1118
          5       0.63      0.45      0.53       667
          6       0.64      0.73      0.68      1389
          7       0.84      0.89      0.86       998
          8       0.74      0.51      0.61       550
          9       0.62      0.48      0.54      1539
         10       0.70      0.69      0.70       881
         11       0.70      0.61      0.65       778
         12       0.95      0.12      0.21       363
         13       0.84      0.80      0.82      1321
         14       0.78      0.75      0.77       926
         15       1.00      0.39      0.57        66
         16       0.73      0.86      0.79 

**SVM classifier**

In [43]:
from sklearn import svm
lin_svm_model = svm.LinearSVC()
lin_svm_model.fit(X_train, y_train) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [44]:
from sklearn.metrics import classification_report,accuracy_score,cohen_kappa_score
y_test_pred = lin_svm_model.predict(X_test)
acc = accuracy_score(y_test_class,y_test_pred)
kap = cohen_kappa_score(y_test_class,y_test_pred)
print("accuracy:",acc)
print("kappa:",kap)
print(classification_report(y_test_class,y_test_pred))

accuracy: 0.7034104107274839
kappa: 0.6832999746660589
             precision    recall  f1-score   support

          0       0.65      0.71      0.68      1593
          1       0.50      0.35      0.41       644
          2       0.65      0.73      0.69      1547
          3       0.74      0.73      0.73      1009
          4       0.68      0.73      0.70      1118
          5       0.57      0.56      0.57       667
          6       0.72      0.74      0.73      1389
          7       0.85      0.89      0.87       998
          8       0.60      0.60      0.60       550
          9       0.61      0.57      0.59      1539
         10       0.71      0.71      0.71       881
         11       0.68      0.73      0.70       778
         12       0.54      0.32      0.40       363
         13       0.85      0.83      0.84      1321
         14       0.73      0.69      0.71       926
         15       0.56      0.27      0.37        66
         16       0.75      0.81      0.78 

In [47]:
with open("trained_first_model.pickle","wb") as file:
  pickle.dump((lr_cv,rf_model,lin_svm_model),file)

In [48]:
pd_train.shape

(91760, 7)

In [49]:
y_train.shape

(91760,)